<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/AGENT_ORCHESTRARION_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet



#FlashAttention only supports Ampere GPUs or newer. #NEED A100 OR L4 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

!pip install crewai --quiet
!pip install langchain --quiet
!pip install faiss-gpu --quiet
!pip install sentence-transformers --quiet

!pip install langchain_community -q

!pip install faiss-cpu -q

!pip install crewai-tools -q

!pip install transformers -U -q

In [ ]:
!nvidia-smi

Sat Mar 22 11:20:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.callbacks.manager import CallbackManager
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import warnings
warnings.filterwarnings("ignore")

import colab_env
from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

## new code

In [ ]:
import logging
import torch
from typing import List, Optional
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
# from langchain_huggingface import HuggingFaceEmbeddings as HFEmbeddings # corrected import

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Utility Functions
def concatenate_documents(docs: List) -> str:
    """
    Concatenates the page content of documents with newline characters.

    Args:
        docs: A list of documents.

    Returns:
        A string containing the concatenated page content.
    """
    try:
        if not docs:
            logging.warning("No documents provided for concatenation.")
            return ""
        return "\n".join([doc.page_content for doc in docs])
    except Exception as e:
        logging.exception("Error concatenating documents:")
        return ""  # Or raise an exception, depending on your error handling policy

# LLM Setup
def setup_mistral_pipeline() -> Optional[HuggingFacePipeline]:
    """
    Sets up the Mistral pipeline with 4-bit quantization.

    Returns:
        A HuggingFacePipeline using the Mistral model, or None if setup fails.
    """
    try:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        model_id = "mistralai/Mistral-7B-Instruct-v0.1" # 01 march 2024 AND
        mistral_pipeline = pipeline(
            "text-generation",
            model=model_id,
            quantization_config=quantization_config
        )
        return HuggingFacePipeline(pipeline=mistral_pipeline)
    except AttributeError as e:
        logging.error(f"AttributeError in setup_mistral_pipeline: {e}")
        return None
    except Exception as e:
        logging.exception("Error setting up Mistral pipeline:")
        return None

def setup_llama2_pipeline() -> Optional[HuggingFacePipeline]:
    """
    Sets up the Llama 2 pipeline.

    Returns:
        A HuggingFacePipeline using the Llama 2 model, or None if setup fails.
    """
    try:
        llama2_model_id = "meta-llama/Llama-2-7b-chat-hf"
        model = AutoModelForCausalLM.from_pretrained(
            llama2_model_id,
            trust_remote_code=True,
            quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        )
        tokenizer = AutoTokenizer.from_pretrained(llama2_model_id, trust_remote_code=True)
        llama2_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            # quantization_config=quantization_config # NameError: name 'quantization_config' is not defined
        )
        return HuggingFacePipeline(pipeline=llama2_pipeline)
    except AttributeError as e:
        logging.error(f"AttributeError in setup_llama2_pipeline: {e}")
        return None
    except Exception as e:
        logging.exception("Error setting up Llama 2 pipeline:")
        return None

# Tools
class ProductInfoRetrieverTool(BaseTool):
    """
    A tool for retrieving product information from a retriever.
    """

    name: str = "Product Information Retriever"
    description: str = "Useful for retrieving product information from internal databases."

    def __init__(self, retriever: RetrievalQA, mistral_pipeline: Optional[callable] = None):
        super().__init__()
        self.retriever = retriever
        self.mistral_pipeline = mistral_pipeline

    def _run(self, query: str) -> str:
        """
        Retrieves product information based on the query.

        Args:
            query: The query to retrieve product information for.

        Returns:
            The retrieved product information.
        """
        if not query:
            logging.error("Query cannot be empty.")
            return "No query provided."

        try:
            docs = self.retriever.get_relevant_documents(query)
            if not docs:
                logging.warning("No relevant documents found.")
                return "No product information found for the given query."

            context = concatenate_documents(docs)  # Use the utility function

            if self.mistral_pipeline:
                try:
                    response = self.mistral_pipeline(context, max_new_tokens=256)
                    content = response[0]['generated_text']
                    return content
                except Exception as e:
                    logging.error(f"Error during mistral pipeline execution: {e}")
                    return "Error processing retrieved information."
            else:
                logging.warning("Mistral pipeline not provided. Returning raw context.")
                return context  # Return the raw context if no pipeline
        except Exception as e:
            logging.exception("Error during product information retrieval:")
            return "An error occurred during product information retrieval."

# Agents
class RetrievalAgent(Agent):
    """
    An agent responsible for retrieving product information.
    """

    def __init__(self, llm: HuggingFacePipeline, retriever_tool: BaseTool): # changed type hint
        super().__init__(
            role='Product Information Retriever',
            goal='Retrieve product details from internal databases.',
            backstory='You are an expert in accessing product specifications.',
            tools=[retriever_tool],
            llm=llm
        )

class SearchAgent(Agent):
    """
    An agent responsible for searching the web for information.
    """
    def __init__(self, llm: HuggingFacePipeline, llm_config: Optional[dict] = None):
        super().__init__(
            role='Web Searcher',
            goal='Gather relevant information from external web sources.',
            backstory='You are an expert at searching the web for information.',
            llm=llm,
            llm_config=llm_config or {"model": "meta-llama/Llama-2-7b-chat-hf", "provider": "huggingface"}
        )

class SentimentAnalyzerAgent(Agent):
    """
    An agent responsible for analyzing sentiment.
    """
    def __init__(self, llm: HuggingFacePipeline, llm_config: Optional[dict] = None):
        super().__init__(
            role='Sentiment Analyzer',
            goal='Determine the customer sentiment.',
            backstory='You are an expert at analyzing customer emotions.',
            llm=llm,
            llm_config=llm_config or {"model": "meta-llama/Llama-2-7b-chat-hf", "provider": "huggingface"}
        )

class ResponseGeneratorAgent(Agent):
    """
    An agent responsible for generating a comprehensive response.
    """
    def __init__(self, llm: HuggingFacePipeline, llm_config: Optional[dict] = None):
        super().__init__(
            role='Response Generator',
            goal='Formulate a comprehensive and helpful response, integrating product information, web search results, and sentiment analysis.',
            backstory='You are an expert at combining retrieved information into a coherent and helpful answer.',
            llm=llm,
            llm_config=llm_config or {"model": "meta-llama/Llama-2-7b-chat-hf", "provider": "huggingface"}
        )

# Tasks
def create_tasks(retrieval_agent: RetrievalAgent, search_agent: SearchAgent, sentiment_agent: SentimentAnalyzerAgent, rag_agent: ResponseGeneratorAgent):
    """
    Creates the tasks for the crew.

    Args:
        retrieval_agent: The retrieval agent.
        search_agent: The search agent.
        sentiment_agent: The sentiment analyzer agent.
        rag_agent: The response generator agent.

    Returns:
        A list of tasks.
    """

    retrieval_task = Task(
        description="Retrieve product information based on the user's query.",
        agent=retrieval_agent,
        expected_output="str"
    )

    search_task = Task(
        description="Search the web for any related information.",
        agent=search_agent,
        expected_output="str"
    )

    sentiment_task = Task(
        description="Determine the sentiment of the customer's query.",
        agent=sentiment_agent,
        expected_output="str"
    )

    rag_task = Task(
        description="""
            Combine the product information, web search results, and sentiment analysis to generate a comprehensive response.
            The response should be structured as follows:

            Product Information: (Summarize the product details)

            Web Search Summary: (Summarize relevant information found on the web)

            Sentiment Analysis: (Report the overall customer sentiment)

            Final Answer: (A comprehensive and helpful answer to the user's query)
            """,
        agent=rag_agent,
        expected_output="str"
    )

    return [retrieval_task, search_task, sentiment_task, rag_task]

# Main Function
def main():
    """
    Orchestrates the crew of agents to handle product-related queries.
    """
    try:
        # 1. Initialize LLMs and Embeddings
        mistral_llm = setup_mistral_pipeline()
        if not mistral_llm:
            logging.error("Failed to setup Mistral pipeline. Exiting.")
            return

        llama2_llm = setup_llama2_pipeline()
        if not llama2_llm:
            logging.error("Failed to setup Llama2 pipeline. Exiting.")
            return

        embeddings = HuggingFaceEmbeddings() # corrected instantiation

        # 2. Setup Vectorstore and Retriever (Example)
        vectorstore = FAISS.from_texts(["Example product information", "More example data"], embeddings)
        retriever = vectorstore.as_retriever()

        # 3. Initialize Tools
        retriever_tool = ProductInfoRetrieverTool(retriever=retriever, mistral_pipeline=mistral_llm)

        # 4. Initialize Agents
        retrieval_agent = RetrievalAgent(llm=mistral_llm, retriever_tool=retriever_tool)
        search_agent = SearchAgent(llm=llama2_llm)
        sentiment_agent = SentimentAnalyzerAgent(llm=llama2_llm)
        rag_agent = ResponseGeneratorAgent(llm=llama2_llm)

        # 5. Create Tasks
        tasks = create_tasks(retrieval_agent, search_agent, sentiment_agent, rag_agent)

        # 6. Create and Run Crew
        crew = Crew(
            agents=[retrieval_agent, search_agent, sentiment_agent, rag_agent],
            tasks=tasks,
            process=Process.sequential  # Or Process.hierarchical
        )

        result = crew.kickoff()
        print("Final result:", result)

    except Exception as e:
        logging.exception("An error occurred during the crew execution:")

if __name__ == "__main__":
    main()